In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import keras
import tensorflow as tf
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

from keras import backend as K
print("GPU:", K.tensorflow_backend._get_available_gpus())

# from tensorflow.python.client import device_lib

# def get_available_gpus():
#     local_device_protos = device_lib.list_local_devices()
#     return [x.name for x in local_device_protos if x.device_type == 'GPU']

#get_available_gpus()

Using TensorFlow backend.




GPU: ['/job:localhost/replica:0/task:0/device:GPU:0']


In [3]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# To print all the output in cells (Deactivate if not necesssary. This is to avoid print() statements in cells)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
listing_df = pd.read_csv("data/listings.csv")
reviews_df = pd.read_csv("data/reviews.csv")
calendar_df = pd.read_csv("data/calendar.csv")

In [5]:
listing_df.head(2)

reviews_df.head(2)

calendar_df.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...


,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00


In [6]:
## Uncomment and run this only once

import spacy
from spacy.tokenizer import Tokenizer

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


nlp = spacy.load('en_core_web_sm')
tokenizer = Tokenizer(nlp.vocab)

def tokenize_docs(df):
    texts = df.comments.apply(str).apply(lambda x: clean_text(x)).tolist()
    tokenized_docs = []
    for i, doc in enumerate(tokenizer.pipe(texts, batch_size=1000)):
        tokenized_docs.append([t.text for t in doc if len(t.text.strip())>0])
    return tokenized_docs


tokenized_reviews = tokenize_docs(reviews_df)
reviews_df["comments_t"] = pd.Series(tokenized_reviews).values
reviews_df.to_json("data/reviews_tokenized.json")

reviews_df = pd.read_json("data/reviews_tokenized.json")

In [7]:
reviews_df.head(2)

,listing_id,id,date,reviewer_id,reviewer_name,comments,comments_t
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...,"[Cute, and, cozy, place, ., Perfect, location,..."
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...,"[Kelly, has, a, great, room, in, a, very, cent..."


In [8]:
import gensim
import codecs

class MySentences(object):
    def __init__(self, dframe):
        self.dframe = dframe

    def __iter__(self):
        for row in self.dframe.iterrows():
            yield [x.lower() for x in row[1]['comments_t']]


def main(df):
    sentences = MySentences(df)
    model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4, )
    model.save("data/w2v_embeddings")


#uncomment this to run word2vec
main(reviews_df)

In [9]:
# check the w2v model
w2v_model = gensim.models.Word2Vec.load("data/w2v_embeddings")
w2v_model.wv.most_similar("rooms")

[('bedrooms', 0.8216997385025024),
 ('bathrooms', 0.7414470911026001),
 ('spaces', 0.7390960454940796),
 ('quarters', 0.672817587852478),
 ('units', 0.6720730066299438),
 ('bedroom', 0.6579513549804688),
 ('room', 0.6351318359375),
 ('accommodations', 0.6175037622451782),
 ('levels', 0.6049386262893677),
 ('beds', 0.5957279205322266)]

### lets get the data ready for some deep learning

In [10]:
import re
import operator
import codecs

num_regex = re.compile('^[+-]?[0-9]+\.?[0-9]*$')

def is_number(token):
    return bool(num_regex.match(token))

def create_vocab(dframe, maxlen=0, vocab_size=0):
    total_words, unique_words = 0, 0
    word_freqs = {}
    top = 0
    for row in dframe.iterrows():
        words = [x.lower() for x in row[1]['comments_t']]
        if maxlen > 0 and len(words) > maxlen:
            continue

        for w in words:
            if not is_number(w):
                try:
                    word_freqs[w] += 1
                except KeyError:
                    unique_words += 1
                    word_freqs[w] = 1
                total_words += 1

    print ('   %i total words, %i unique words' % (total_words, unique_words))
    sorted_word_freqs = sorted(word_freqs.items(), key=operator.itemgetter(1), reverse=True)

    vocab = {'<pad>':0, '<unk>':1, '<num>':2}
    index = len(vocab)
    for word, _ in sorted_word_freqs:
        vocab[word] = index
        index += 1
        if vocab_size > 0 and index > vocab_size + 2:
            break
    if vocab_size > 0:
        print ('  keep the top %i words' % vocab_size)

    #Write (vocab, frequence) to a txt file
    vocab_file = codecs.open('data/vocab.txt', mode='w', encoding='utf8')
    sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1))
    for word, index in sorted_vocab:
        if index < 3:
            vocab_file.write(word+'\t'+str(0)+'\n')
            continue
        vocab_file.write(word+'\t'+str(word_freqs[word])+'\n')
    vocab_file.close()

    return vocab


def read_data(dframe, maxlen=0):
    num_hit, unk_hit, total = 0., 0., 0.
    maxlen_x = 0
    data_x = []
    for row in dframe.iterrows():
        words = [x.lower() for x in row[1]['comments_t']]
        if maxlen > 0 and len(words) > maxlen:
            continue

        indices = []
        for word in words:
            if is_number(word):
                indices.append(vocab['<num>'])
                num_hit += 1
            elif word in vocab:
                indices.append(vocab[word])
            else:
                indices.append(vocab['<unk>'])
                unk_hit += 1
            total += 1

        data_x.append(indices)
        if maxlen_x < len(indices):
            maxlen_x = len(indices)

    print('   <num> hit rate: %.2f, <unk> hit rate: %.2f' % (100*num_hit/total, 100*unk_hit/total))
    return data_x, maxlen_x


In [11]:
vocab = create_vocab(reviews_df)
train_data, max_length = read_data(reviews_df)

   6745037 total words, 40378 unique words
   <num> hit rate: 0.41, <unk> hit rate: 0.00


In [12]:
from keras.preprocessing import sequence

train_x = sequence.pad_sequences(train_data, maxlen=max_length)

print('Number of training examples: ', len(train_x))
print('Length of vocab: ', len(vocab))

def sentence_batch_generator(data, batch_size):
    n_batch = len(data) / batch_size
    batch_count = 0
    np.random.shuffle(data)

    while True:
        if batch_count == n_batch:
            np.random.shuffle(data)
            batch_count = 0

        batch = data[batch_count*batch_size: (batch_count+1)*batch_size]
        batch_count += 1
        yield batch

def negative_batch_generator(data, batch_size, neg_size):
    data_len = data.shape[0]
    dim = data.shape[1]

    while True:
        indices = np.random.choice(data_len, batch_size * neg_size)
        samples = data[indices].reshape(batch_size, neg_size, dim)
        yield samples

Number of training examples:  84849
Length of vocab:  40381


In [13]:
# Setting some configs for the training run
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, 
                                  epsilon=1e-08, clipnorm=10, clipvalue=0)

In [14]:
from aspectmodel import create_model
import keras.backend as K
import logging

logging.basicConfig(
                    #filename='out.log',
                    level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger(__name__)

logger.info('  Building model')


def max_margin_loss(y_true, y_pred):
    return K.mean(y_pred)

model = create_model(max_length, vocab, "data/w2v_embeddings")
# freeze the word embedding layer
model.get_layer('word_emb').trainable=False
model.compile(optimizer=optimizer, loss=max_margin_loss, metrics=[max_margin_loss])

2020-02-05 16:34:52,798 INFO   Building model


2020-02-05 16:34:52,804 WARNING From /home/sidd/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2020-02-05 16:34:52,809 WARNING From /home/sidd/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



2020-02-05 16:34:52,855 WARNING From /home/sidd/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/home/sidd/projects/airbnb_seattle_submission/aspectmodel.py:237: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ma...)`
  model = Model(input=[sentence_input, neg_input], output=loss)
2020-02-05 16:34:53,641 INFO Loading embeddings from: data/w2v_embeddings
2020-02-05 16:34:53,644 INFO loading Word2Vec object from data/w2v_embeddings
2020-02-05 16:34:54,027 INFO loading wv recursively from data/w2v_embeddings.wv.* with mmap=None
2020-02-05 16:34:54,031 INFO setting ignored attribute vectors_norm to None
2020-02-05 16:34:54,033 INFO loading vocabulary recursively from data/w2v_embeddings.vocabulary.* with mmap=None
2020-02-05 16:34:54,034 INFO loading trainables recursively from data/w2v_embeddings.trainables.* 

2020-02-05 16:34:59,240 WARNING From /home/sidd/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.



2020-02-05 16:35:00,097 WARNING From /home/sidd/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.

2020-02-05 16:35:02,334 INFO 12094/40381 word vectors initialized (hit rate: 29.95%)
2020-02-05 16:35:02,461 INFO Initializing aspect embedding matrix as centroid of kmean clusters


2020-02-05 16:35:10,313 WARNING From /home/sidd/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [15]:
## Training
#
from keras.models import load_model
from tqdm import tqdm_notebook as tqdm
from time import time

logger.info('--------------------------------------------------------------------------------------------------------------------------')

vocab_inv = {}
for w, ind in vocab.items():
    vocab_inv[ind] = w



batches_per_epoch = len(train_x) // 32
batches_per_epoch = 500

min_loss = float('inf')
for ii in range(10):
    sen_gen = sentence_batch_generator(train_x ,32)
    neg_gen = negative_batch_generator(train_x, 32, 5)
    t0 = time()
    loss, max_margin_loss = 0., 0.

    for b in tqdm(range(batches_per_epoch)):
        sen_input = next(sen_gen) 
        neg_input = next(neg_gen)
        neg_input = neg_input[:len(sen_input)]
        l = len(sen_input) #+ len(neg_input)
        batch_loss, batch_max_margin_loss = model.train_on_batch([sen_input, neg_input], np.ones((l, 1)))
        #print(batch_loss, batch_max_margin_loss)
        loss += batch_loss / batches_per_epoch
        max_margin_loss += batch_max_margin_loss / batches_per_epoch

    tr_time = time() - t0

    if loss< min_loss:
        min_loss = loss
        word_emb = model.get_layer('word_emb').get_weights()[0]
        aspect_emb = model.get_layer('aspect_emb').get_weights()[0]
        word_emb = word_emb / np.linalg.norm(word_emb, axis=-1, keepdims=True)
        aspect_emb = aspect_emb / np.linalg.norm(aspect_emb, axis=-1, keepdims=True)
        aspect_file = codecs.open('output/aspect.log', 'w', 'utf-8')
        model.save_weights('output/model_param')

        for ind in range(len(aspect_emb)):
            desc = aspect_emb[ind]
            sims = word_emb.dot(desc.T)
            ordered_words = np.argsort(sims)[::-1]
            desc_list = [vocab_inv[w] for w in ordered_words[:10]]
            print('Aspect %d:' % ind)
            print(desc_list)
            aspect_file.write('Aspect %d:\n' % ind)
            aspect_file.write(' '.join(desc_list) + '\n\n')

    logger.info('Epoch %d, train: %is' % (ii, tr_time))
    logger.info('Total loss: %.4f, max_margin_loss: %.4f, ortho_reg: %.4f' % (loss, max_margin_loss, loss-max_margin_loss))
            

2020-02-05 16:35:10,418 INFO --------------------------------------------------------------------------------------------------------------------------


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2020-02-05 16:35:10,929 WARNING From /home/sidd/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2020-02-05 16:35:12,724 WARNING From /home/sidd/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



2020-02-05 16:35:12,891 WARNING From /home/sidd/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



...

2020-02-05 16:36:22,740 INFO Epoch 8, train: 6s
2020-02-05 16:36:22,745 INFO Total loss: 0.7972, max_margin_loss: 0.6936, ortho_reg: 0.1036


Aspect 2:
['thorough', 'communicative', 'prompt', 'responsive', 'understanding', 'proactive', 'timely', 'informative', 'helpful', 'promptly']
Aspect 3:
['stadiums', 'campus', 'waterfront', 'stadium', 'south', 'northgate', 'chinatown', 'buses', 'uw', 'broadway']
Aspect 4:
['return', 'book', 'come', 'again', 'go', 'continue', 'choose', 'visit', 'surely', 'love']
Aspect 5:
['terry', 'patrick', 'laura', 'paul', 'sean', 'steve', 'cindy', 'megan', 'susanna', 'kate']
Aspect 6:
['are', 'were', 'surprisingly', 'roomy', 'attractive', 'functional', 'spacious', 'tasteful', 'stylish', 'cosy']
Aspect 7:
['breaker', 'why', 'dont', 'fancy', 'honestly', 'otherwise', 'fair', 'nothing', 'should', 'probably']
Aspect 8:
['scheduled', 'dark', 'until', 'august', 'usually', 'past', 'twice', '6pm', 'plane', 'because']
Aspect 9:
['fabulous', 'fantastic', 'loved', 'terrific', 'wonderful', 'brilliant', 'phenomenal', 'great', 'superb', 'awesome']



Aspect 0:
['duties', 'rambunctious', 'asia', 'beim', 'solidly', 'pound', 'tape', 'othello', 'willkommen', 'graced']
Aspect 1:
['beverages', 'drinks', 'snack', 'teas', 'snacks', 'refreshments', 'beer', 'goodies', 'treats', 'condiments']
Aspect 2:
['thorough', 'communicative', 'prompt', 'responsive', 'understanding', 'proactive', 'informative', 'timely', 'helpful', 'promptly']
Aspect 3:
['stadiums', 'campus', 'waterfront', 'stadium', 'south', 'northgate', 'chinatown', 'buses', 'uw', 'pier']
Aspect 4:

2020-02-05 16:36:30,690 INFO Epoch 9, train: 7s
2020-02-05 16:36:30,691 INFO Total loss: 0.7797, max_margin_loss: 0.6768, ortho_reg: 0.1029



['return', 'book', 'come', 'again', 'go', 'continue', 'choose', 'visit', 'surely', 'love']
Aspect 5:
['terry', 'patrick', 'paul', 'laura', 'cindy', 'sean', 'steve', 'susanna', 'megan', 'kate']
Aspect 6:
['are', 'were', 'surprisingly', 'attractive', 'roomy', 'spacious', 'functional', 'tasteful', 'stylish', 'cosy']
Aspect 7:
['breaker', 'why', 'fancy', 'dont', 'honestly', 'otherwise', 'fair', 'nothing', 'probably', 'strong']
Aspect 8:
['august', 'scheduled', 'until', 'dark', 'usually', 'past', '6pm', 'plane', 'twice', 'flight']
Aspect 9:
['fabulous', 'fantastic', 'loved', 'terrific', 'wonderful', 'brilliant', 'phenomenal', 'great', 'superb', 'awesome']


In [16]:
# lets get top k words per aspect
from collections import defaultdict
TOP_K = 100
aspect2words = defaultdict(list)
word_emb = model.get_layer('word_emb').get_weights()[0]
aspect_emb = model.get_layer('aspect_emb').get_weights()[0]
word_emb = word_emb / np.linalg.norm(word_emb, axis=-1, keepdims=True)
aspect_emb = aspect_emb / np.linalg.norm(aspect_emb, axis=-1, keepdims=True)

for ind in range(len(aspect_emb)):
    desc = aspect_emb[ind]
    sims = word_emb.dot(desc.T)
    ordered_words = np.argsort(sims)[::-1]
    aspect2words[ind] = set([vocab_inv[w] for w in ordered_words[:TOP_K]])

In [17]:
w2aspects = defaultdict(list)
for aspect, words in aspect2words.items():
    for word in words:
        w2aspects[word].append(aspect)


In [18]:
# lets go back to reviews and see what people are talking about for a listing. 
# top reviewed in this dataset
reviews_df['listing_id'].value_counts()[100:200]

1773803    124
3615614    124
2660384    123
251922     122
817115     121
          ... 
3652434     88
1455269     88
4118282     88
1651324     88
3293438     88
Name: listing_id, Length: 100, dtype: int64

In [19]:
def getaspect2reviews(f_rvws, a2w, w2a):
    #print(f_rvws)
    score = np.zeros((len(f_rvws), len(a2w)))
    a2indices = []
    for i, review in enumerate(f_rvws):
        words = [x.lower() for x in review]
        a2word_pos = defaultdict(list)
        cur_word_idx = -100
        for word_pos, word in enumerate(words):
            indices = w2a.get(word, [])
            score[i, indices]+=1
            for j in indices:
#                 if not word_pos-cur_word_idx<=10:
                a2word_pos[j].append(word_pos)
#                     cur_word_idx = word_pos
        a2indices.append(a2word_pos)
    return score, a2indices
        
    

In [20]:
import random

user_aspect = 3
number_of_reviews = 5
listing_id = 1023693

In [21]:
f_rvws = reviews_df[reviews_df.listing_id == listing_id].comments_t
scores, a2indices = getaspect2reviews(f_rvws, aspect2words, w2aspects)
top_indices = np.argsort(scores[:, user_aspect])[::-1][:number_of_reviews]

print("Reviews on aspect: {} for listing {}".format(user_aspect, listing_id))
for review_idx in top_indices:
    if len(a2indices[review_idx][user_aspect])>0:
        word_pos = random.choice(a2indices[review_idx][user_aspect])
        review_txt = f_rvws.iloc[review_idx]
        print("Review:", review_idx)
        print(" ".join(review_txt[word_pos-15:word_pos] + review_txt[word_pos:word_pos + 15]))
    

Reviews on aspect: 3 for listing 1023693
Review: 1
Ballard was such a treat ! We were looking for something outside downtown Seattle ,
Review: 94
midnight . We also really liked the detailed rules and suggestions ( for bars , restaurants , etc . ) she left for us . The apartment is quiet and
Review: 118
is walkable to so many good things : wine shops , restaurants , parks , grocery stores . Unlike Old Ballard , the neighborhood is also easily accessible from elsewhere
Review: 91
restaurants very close to the apartment , and downtown Ballard has tons of restaurants and cafes ! I loved it here .
Review: 10
decorated and in a quiet neighborhood within walking distance to Ballard ' s bars , restaurants , etc . ( and a quick drive to downtown Seattle ) . The
